Challenge: We’ve observed an increase in suspected fake reviews on our platform. These reviews
mislead consumers and damage our brand reputation. We need a systematic way to identify
potentially fake reviews for further investigation.
Request: Develop a classification system that can flag potentially artificial or inauthentic reviews. The system should:
• Identify patterns characteristic of fake reviews
• Have a low false positive rate to avoid flagging legitimate reviews
• Provide explainable results that our team can use for investigations
Success metrics: High precision in identifying suspicious reviews and clear explanation of the
patterns that trigger flags

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report



In [ ]:
# Load real cleaned reviews

real_reviews = pd.read_pickle("/content/drive/MyDrive/DEAssignment3/review_review_df_with_clean_text.pkl")

# Checking first few rows
print(f"Loaded {len(real_reviews)} real reviews.")
real_reviews.head()


Creating fake reviews

In [ ]:
# Components to build fake reviews
positive_adjectives = [
    "amazing", "fantastic", "incredible", "perfect", "outstanding", "good", "great", "fabulous",
    "heartwarming", "brilliant", "exceptional", "lovely", "cute", "adorable", "touching", "funny",
    "gorgeous", "spectacular", "breathtaking", "remarkable", "impressive", "charming", "delightful",
    "memorable", "beautiful", "stellar","weird","okay"
]

negative_adjectives = [
    "terrible", "horrible", "disappointing", "poor", "awful", "bad", "gross", "cheap", "flimsy",
    "brutal", "unreliable", "uncomfortable", "mediocre", "careless", "ignorant", "nasty", "finicky",
    "underwhelming", "shoddy", "incomplete", "faulty", "dull", "boring", "broken", "dirty","weird","okay"
]

nouns = [
    "product", "item", "purchase", "experience", "quality", "gift", "thing", "present", "tool",
    "pair", "set", "device", "investment", "choice", "addition", "purchase decision", "gadget",
    "essential", "accessory"
]

positive_verbs = [
    "loved", "enjoyed", "recommended", "appreciated", "valued", "liked", "gifted", "bought",
    "admired", "smiled", "treasured", "swooned", "laughed", "celebrated", "endorsed", "relished",
    "embraced", "wowed by", "was thrilled with","satisfied"
]

negative_verbs = [
    "hated", "disliked", "regretted", "returned", "complained about", "waited", "struggled with",
    "abandoned", "wished I hadn't bought", "misled", "detested", "suffered through", "was disappointed by",
    "fought with", "fumbled through", "was annoyed by", "questioned buying","satisfied"
]

extra_positive_sentences = [
    "Shipping was quick.",
    "Would definitely buy again.",
    "Exceeded my expectations.",
    "This made the perfect gift.",
    "Absolutely worth every penny.",
    "Five stars without hesitation.",
    "Made my day better!",
    "Loved it so much!",
    "Thoroughly enjoyed this a lot.",
    "High quality through and through.",
    "It brought a smile to my face.",
    "Can't stop recommending it!",
    "Feels like a luxury purchase.",
    "Brightened up my whole week.",
    "Everything about it was perfect."
]

extra_negative_sentences = [
    "Not worth the money.",
    "Fell apart after one use.",
    "Extremely disappointed.",
    "Would not recommend to anyone.",
    "Regret buying this.",
    "Waste of time and money.",
    "Very cheaply made.",
    "It was just okay.",
    "Will not be re-purchasing.",
    "Disappointed in this one.",
    "Broke way too easily.",
    "Customer service was awful.",
    "Expected much better for the price.",
    "Would give zero stars if I could.",
    "Never buying from here again."
]

# Function to generate one fake review
def generate_fake_review(sentiment="positive"):
    adj = random.choice(positive_adjectives if sentiment == "positive" else negative_adjectives)
    noun = random.choice(nouns)
    verb = random.choice(positive_verbs if sentiment == "positive" else negative_verbs)

    # 1. Create first sentence - now with more variety
    templates = [
        f"I {verb} this {noun}. It is absolutely {adj}.",
        f"This {noun} was {adj}! I {verb} it.",
        f"Such a {adj} {noun}! I {verb} it.",
        f"I {verb} the {noun}. Truly {adj}.",
        f"My {noun} experience was {adj}.",
        f"I can't believe how {adj} this {noun} was!",
        f"I was really {adj} with this {noun}.",
        f"This {noun} completely {verb} me!",
        f"My experience with this {noun} was simply {adj}.",
        f"I never expected a {noun} to be so {adj}!",
        f"This {noun} exceeded all my expectations. Truly {adj}.",
        f"I {verb} my {noun}, it was {adj}!",
        f"Honestly, this {noun} was just {adj}.",
        f"I found the {noun} to be {adj} overall.",
        f"It's a {adj} {noun}, that's for sure.",
    ]

    # Occasionally (10% chance) drop the adjective for realism
    if random.random() < 0.1:
        templates.append(f"I {verb} this {noun}.")  # Basic simple sentence

    review = random.choice(templates)

    # 2. Optional "but" clause for realism (small chance, ~15%)
    if random.random() < 0.15:
        but_phrases = [
            "but shipping took a while.",
            "but packaging could be better.",
            "but it was smaller than expected.",
            "but setup was a bit tricky.",
            "but color was slightly off.",
            "but it didn't last as long as I hoped.",
            "but it wasn't exactly what I expected.",
            "but it felt a little cheap.",
            "but the instructions were confusing.",
            "but customer service could have been better.",
            "but the price was a little high.",
        ]
        review += " " + random.choice(but_phrases)

    # 3. Add 1–3 extra positive or negative sentences
    if sentiment == "positive":
        extras = random.sample(extra_positive_sentences, k=random.choice([1, 2, 3]))
    else:
        extras = random.sample(extra_negative_sentences, k=random.choice([1, 2, 3]))

    review += " " + " ".join(extras)

    return review


In [ ]:
# Preview 10 random fake reviews
print("\n=== Preview of 10 Generated Fake Reviews ===")
for _ in range(10):
    sentiment = random.choice(["positive", "negative"])
    print(f"({sentiment.title()}) {generate_fake_review(sentiment)}\n")


In [ ]:
#Labeling real reviews and combining

# adding "is_fake" column to real reviews (real = 0)
real_reviews["is_fake"] = 0

# Select only necessary columns (matching structure)
real_reviews_subset = real_reviews[["clean_text", "rating", "parent_asin", "category", "is_fake"]]

# Generate fake reviews
fake_reviews = []

categories = real_reviews["category"].unique()
for category in categories:
    for _ in range(100):  # 100 fake reviews per category
        sentiment = random.choice(["positive", "negative"])
        review_text = generate_fake_review(sentiment=sentiment)
        rating = random.choice([1, 2, 3, 4, 5])
        parent_asin = f"FAKE-{random.randint(10000,99999)}"  # made-up ID

        fake_reviews.append({
            "clean_text": review_text,
            "rating": rating,
            "parent_asin": parent_asin,
            "category": category,
            "is_fake": 1
        })

# Turn into DataFrame
fake_reviews_df = pd.DataFrame(fake_reviews)

# Combine real + fake
combined_reviews = pd.concat([real_reviews_subset, fake_reviews_df], ignore_index=True)

print(f"Combined dataset now has {len(combined_reviews)} reviews.")
combined_reviews.head()


In [ ]:
#Saving fake reviews to a csv:

fake_reviews_df.to_csv("/content/fake_reviews.csv", index=False)

print("✅ Fake reviews saved to fake_reviews.csv")


In [ ]:
# Initialize TF-IDF Vectorizer
tfidf = TfidfVectorizer(max_features=300)

# Fit and transform the review text
X_text = tfidf.fit_transform(combined_reviews["clean_text"])

# Target variable
y = combined_reviews["is_fake"]


In [ ]:
# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_text, y, test_size=0.2, random_state=42, stratify=y
)

print("Training samples:", X_train.shape[0])
print("Testing samples:", X_test.shape[0])


In [ ]:
# Logistic Regression
log_reg = LogisticRegression(max_iter=1000, random_state=42, class_weight = "balanced")
log_reg.fit(X_train, y_train)

# Predict
y_pred_log = log_reg.predict(X_test)

# Evaluation
print("\n=== Logistic Regression Results ===")
print("Accuracy:", accuracy_score(y_test, y_pred_log))
print("Precision:", precision_score(y_test, y_pred_log))
print("Recall:", recall_score(y_test, y_pred_log))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_log))
print(classification_report(y_test, y_pred_log))


In [ ]:
# Random Forest Classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42,class_weight = "balanced")
rf_clf.fit(X_train, y_train)

# Predict
y_pred_rf = rf_clf.predict(X_test)

# Evaluation
print("\n=== Random Forest Classifier Results ===")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Precision:", precision_score(y_test, y_pred_rf))
print("Recall:", recall_score(y_test, y_pred_rf))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))


Performance split out by category
- Predicted probabilities of being fake and hard predictions are made using 0.5 threshold
- Accuracy, precision and recall are calculated for each category and results are compiled in a single df
-Results are visualized in bar plots

In [ ]:
# Add category info back to test set
X_test_indices = X_test.nonzero()[0]
test_categories = combined_reviews.iloc[y_test.index]["category"].values

# Predict probabilities and predictions
y_proba_log = log_reg.predict_proba(X_test)[:, 1]  # probability of being fake
y_pred_log = (y_proba_log > 0.5).astype(int)        # threshold at 0.5

# Analyze performance per category - for each category, assess accuracy, precision and recall of predictions
category_metrics = []

for category in np.unique(test_categories):
    mask = test_categories == category
    y_true_cat = y_test[mask]
    y_pred_cat = y_pred_log[mask]

    accuracy = accuracy_score(y_true_cat, y_pred_cat)
    precision = precision_score(y_true_cat, y_pred_cat, zero_division=0)
    recall = recall_score(y_true_cat, y_pred_cat, zero_division=0)

    category_metrics.append({
        "category": category,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall
    })

# Create results DataFrame
category_results_df = pd.DataFrame(category_metrics).sort_values(by="accuracy", ascending=False)

# Results Display
print("\n=== Performance by Product Category ===")
print(category_results_df)

# Accuracy Plot
plt.figure(figsize=(10,6))
sns.barplot(data=category_results_df, x="accuracy", y="category")
plt.title("Fake Review Detection Accuracy by Category")
plt.xlabel("Accuracy")
plt.ylabel("Product Category")
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,6))
sns.barplot(data=category_results_df, x="precision", y="category")
plt.title("Fake Review Detection Precision by Category")
plt.xlabel("Precision")
plt.ylabel("Product Category")
plt.tight_layout()
plt.show()


In [ ]:
#Feature importance for Logistic Regression
#Get feature names from TF-IDF
feature_names = tfidf.get_feature_names_out()

# Get coefficients - this tells us how influential each feature is
coefs = log_reg.coef_[0]

# Create a DataFrame
coef_df = pd.DataFrame({
    "term": feature_names,
    "coefficient": coefs
})

# Sort by impact greatest to lowest
top_positive = coef_df.sort_values(by="coefficient", ascending=False).head(15)  # words that strongly predict fake
top_negative = coef_df.sort_values(by="coefficient", ascending=True).head(15)   # words that strongly predict real

# 5. Plot
plt.figure(figsize=(12, 6))
sns.barplot(data=pd.concat([top_positive, top_negative]), x="coefficient", y="term", palette="coolwarm")
plt.title("Top Words Predicting Fake vs Real Reviews")
plt.xlabel("Coefficient (positive = more fake, negative = more real)")
plt.ylabel("Term")
plt.tight_layout()
plt.show()

#Printing top ten terms
print("\n=== Top Positive (Fake) Indicators ===")
print(top_positive)

print("\n=== Top Negative (Real) Indicators ===")
print(top_negative)


Threshold Analysis - generate the predicted probability that a review is fake using different thresholds to adjust the sensitivity of the fake review detection system

In [ ]:
# Get predicted probabilities
y_probs = log_reg.predict_proba(X_test)[:, 1]

# Define thresholds
thresholds = [0.3, 0.5, 0.7, 0.9]

# Lists to store metrics
precision_scores = []
recall_scores = []

# Loop through thresholds and store metrics
for thresh in thresholds:
    y_pred_thresh = (y_probs >= thresh).astype(int)

    precision = precision_score(y_test, y_pred_thresh) #correctness of flagged fakes
    recall = recall_score(y_test, y_pred_thresh) #how many true fakes are caught

    precision_scores.append(precision)
    recall_scores.append(recall)

    print(f"\n=== Threshold: {thresh} ===")
    print("Accuracy:", accuracy_score(y_test, y_pred_thresh))
    print("Precision:", precision)
    print("Recall:", recall)
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_thresh))

# 5. Plot precision and recall
plt.figure(figsize=(8,6))
plt.plot(thresholds, precision_scores, marker='o', label='Precision')
plt.plot(thresholds, recall_scores, marker='s', label='Recall')
plt.title('Precision and Recall vs Threshold')
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.xticks(thresholds)
plt.ylim(0, 1.05)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
